In [1]:
#pip install tqdm

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:
BASE_URL = 'https://www.flipkart.com'
SEARCH_QUERY = 'smart watches'
BRANDS=['Noise', 'boAt','Fire-Boltt']
REVIEW_PAGES_TO_SCRAPE = 100

def getSoup(url):
    r =requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup


def getProducts(url):
    product_links = []
    soup = getSoup(url)
    for item in soup.find_all('a', class_='_1fQZEK'):
        link = BASE_URL + item.get('href')
        name = item.find_next('div', class_='_4rR01T')
        product_links.append({'link': link,
                             'name' : name})
    return product_links
 

def getProductData(url):
    soup = getSoup(url)
    m_no = soup.find('td', string='Model Number').find_next_sibling("td").string
    m_name = soup.find('td', string='Model Name').find_next_sibling("td").string
    review_link = BASE_URL + soup.find('div', string='Ratings & Reviews').find_parent('div', class_='_3HKIdy').find_next_sibling('a').get('href')
    reviews = getReviews(review_link)
    d = {'Model_no':m_no,
        'Model_name':m_name,
        'Reviews':reviews}
    return d
    
    
def getReviews(url):
    reviews_list = []
    for i in tqdm(range(1,REVIEW_PAGES_TO_SCRAPE+1)):
        soup = getSoup(url+'&page='+str(i))
        for item in soup.find_all('div', class_='col _2wzgFH K0kLPL'):
            try :
                rating = item.find_next().find_next().text
                review_text = item.find_next('div', class_='t-ZTKy').find_next().find_next().text
                reviewer = item.find_next('p', class_='_2sc7ZR _2V5EHH').text
                city = (item.find_next('p', class_='_2mcZGG').find_next().find_next().text).split(',')[1]
                reviews_list.append({'rating':rating,
                         'review':review_text,
                         'reviewer':reviewer,
                         'city':city})
            except :
                continue
    return reviews_list          

In [4]:
SEARCH_URL = None
for BRAND in BRANDS:
    print('Working for {}\n'.format(BRAND))
    PRODUCTS = []
    p_links = []
    seen = []
    SEARCH_URL = BASE_URL+'/search?q='+SEARCH_QUERY+'&p[]=facets.brand[]='+BRAND
    p_list = getProducts(SEARCH_URL)
    
    #remove duplicate products
    for p in p_list:
        if p.get('name') not in seen:
            seen.append(p.get('name'))
            p_links.append(p.get('link'))
    
    for link in p_links:
        d = getProductData(link)
        for review in d.get('Reviews'):  
            product = {'Brand' : BRAND,
                        'Model_no' : d.get('Model_no'),
                        'Model_name' : d.get('Model_name'),
                        'Reviewer' : review.get('reviewer'),
                        'City' : review.get('city'),
                        'Rating' : review.get('rating'),
                        'Review' : review.get('review') }
            PRODUCTS.append(product) 
            
    print('Writing to file..')
    df = pd.DataFrame(PRODUCTS)
    if(BRAND =='Noise'):
        df.to_csv('Reviews_Noise.csv')
    elif(BRAND =='boAt'):
        df.to_csv('Reviews_boAt.csv')
    elif(BRAND =='Fire-Boltt'):
        df.to_csv('Reviews_FireBoltt.csv') 
    

Working for Noise



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:07<00:00,  1.27s/it]


Writing to file..
Working for boAt



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:46<00:00,  1.07s/it]


Writing to file..
Working for Fire-Boltt



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:24<00:00,  1.19it/s]

Writing to file..
